In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
nltk.download('wordnet')
nltk.download('stopwords')

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk. stem import WordNetLemmatizer

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
# from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

import pickle

%matplotlib inline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
with open("preprocessed_fake_real_data.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

In [3]:
df

,title,content,publication,type,title_len,content_len,title_upercase_len,title_punc_len,content_punc_len,title_uppercase_len,content_uppercase_len,avg_sent_len,content_len_per,content_uppercase_len_per,content_punc_len_per,title_len_per,title_uppercase_len_per,title_punc_len_per,avg_sent_len_per
0,Muslims BUSTED: They Stole Millions In Gov’t B...,muslim busted stole million gov benefit print ...,100percentfedup,fake,53,505,13,51,494,13,12,71.00,3.611312e-04,0.000271,3.638231e-04,0.002902,0.006130,0.002869,0.001778
1,Re: Why Did Attorney General Loretta Lynch Ple...,attorney general loretta lynch plead fifth att...,100percentfedup,fake,59,1789,10,57,1736,10,68,147.83,1.279334e-03,0.001536,1.278536e-03,0.003230,0.004715,0.003207,0.003703
2,BREAKING: Weiner Cooperating With FBI On Hilla...,breaking weiner cooperating fbi hillary email ...,100percentfedup,fake,68,1363,18,67,1323,18,37,150.44,9.746968e-04,0.000836,9.743682e-04,0.003723,0.008488,0.003769,0.003768
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,pin drop speech father daughter kidnapped kill...,100percentfedup,fake,125,271,44,118,265,44,14,66.75,1.937952e-04,0.000316,1.951682e-04,0.006844,0.020747,0.006639,0.001672
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,fantastic trump point plan reform healthcare b...,100percentfedup,fake,99,2095,32,94,2058,32,91,109.11,1.498158e-03,0.002056,1.515684e-03,0.005421,0.015089,0.005288,0.002733
5,Hillary Goes Absolutely Berserk On Protester A...,hillary go absolutely berserk protester rally ...,100percentfedup,fake,62,1255,9,59,1184,9,69,103.50,8.974648e-04,0.001559,8.719970e-04,0.003395,0.004244,0.003319,0.002592
6,BREAKING! NYPD Ready To Make Arrests In Weiner...,breaking nypd ready make arrest weiner case hi...,100percentfedup,fake,223,5870,38,213,5658,38,239,176.48,4.197704e-03,0.005400,4.167026e-03,0.012210,0.017918,0.011983,0.004421
7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,wow whistleblower tell chilling story massive ...,100percentfedup,fake,173,2521,57,167,2443,57,156,156.25,1.802796e-03,0.003525,1.799230e-03,0.009472,0.026877,0.009395,0.003914
8,BREAKING: CLINTON CLEARED...Was This A Coordin...,breaking clinton cleared coordinated last minu...,100percentfedup,fake,118,1294,34,110,1265,34,70,183.14,9.253541e-04,0.001582,9.316521e-04,0.006461,0.016032,0.006189,0.004587
9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",evil hillary supporter yell f ck trump burn tr...,100percentfedup,fake,143,727,43,134,708,43,49,89.50,5.198860e-04,0.001107,5.214306e-04,0.007830,0.020276,0.007539,0.002242


In [4]:
y = pd.get_dummies(df['type']) # fake, real:two column one-hot encoding
y = np.array(y)

In [5]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [15]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 60000
# Max number of words in each news.
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df['content'].values)
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 155592 unique tokens.


In [16]:
X = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of X:', X.shape)

Shape of X: (28665, 500)


In [17]:
#Adding title length into tf-itf vectorize matrix
X = np.concatenate((X, df['title_len'].values.reshape(-1,1)), axis=1)
X

array([[    0,     0,     0, ...,   199,   730,    53],
       [    0,     0,     0, ...,  2334,   730,    59],
       [    0,     0,     0, ...,   740,   730,    68],
       ...,
       [  160,   291, 10328, ...,   190, 17984,   105],
       [    0,     0,     0, ...,     3,   299,    60],
       [    0,     0,     0, ...,   375,  2468,    79]])

In [18]:
print('Shape of X after adding title length as features:', X.shape)

Shape of X after adding title length as features: (28665, 501)


In [19]:
Y = pd.get_dummies(df['type']).values
 
# Y = pd.get_dummies(df['type'])
# Y = Y.iloc[:,1].values

print('Shape of Y:', Y.shape)

Shape of Y: (28665, 2)


In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(22932, 501) (22932, 2)
(5733, 501) (5733, 2)


In [21]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.4)) # recurrent_dropout=0.2
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 501, 100)          6000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 130       
Total params: 6,129,794
Trainable params: 6,129,794
Non-trainable params: 0
_________________________________________________________________


In [22]:
es = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=0, mode='auto')

model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), callbacks=[es])

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 22932 samples, validate on 5733 samples
Epoch 1/100
22932/22932 [==============================] - 243s 11ms/step - loss: 0.3483 - accuracy: 0.8450 - val_loss: 0.2085 - val_accuracy: 0.9224
Epoch 2/100
22932/22932 [==============================] - 241s 11ms/step - loss: 0.1319 - accuracy: 0.9519 - val_loss: 0.1954 - val_accuracy: 0.9339
Epoch 3/100
22932/22932 [==============================] - 242s 11ms/step - loss: 0.0828 - accuracy: 0.9733 - val_loss: 0.2200 - val_accuracy: 0.9233
Epoch 4/100
22932/22932 [==============================] - 241s 11ms/step - loss: 0.0460 - accuracy: 0.9837 - val_loss: 0.2891 - val_accuracy: 0.9271
Epoch 5/100
22932/22932 [==============================] - 241s 10ms/step - loss: 0.0215 - accuracy: 0.9927 - val_loss: 0.3412 - val_accuracy: 0.9301


In [23]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
print("train-acc = " + str(accuracy_score(np.argmax(Y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(test_pred, axis=1))))

train-acc = 0.9970347113204256
test-acc = 0.9300540729112158
